In [1]:
pip install torch torchvision pandas matplotlib


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\user\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [8]:
os.getcwd()

'C:\\Users\\user\\Downloads\\SelebA'

In [6]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch import nn, optim
from PIL import Image
import numpy as np

# Пути к данным CelebA
DATASET_PATH = os.getcwd()  # Укажите путь к датасету
IMAGES_PATH = os.path.join(DATASET_PATH, "img_align_celeba/img_align_celeba")
ATTRIBUTES_PATH = os.path.join(DATASET_PATH, "list_attr_celeba.csv")

# Загрузка атрибутов
attributes = pd.read_csv(ATTRIBUTES_PATH)
attributes.replace(-1, 0, inplace=True)  # Преобразуем -1 в 0 для удобства

# Оставляем только интересующие атрибуты (например, Smiling, Heavy_Makeup)
#TARGET_ATTRIBUTES = ["Smiling", "Heavy_Makeup"]
#attributes = attributes[TARGET_ATTRIBUTES]


In [7]:
attributes.set_index(attributes.columns[0], inplace=True)

In [8]:
attributes

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
image_id,,,,,,,,,,,,,,,,,,,,,
000001.jpg,0,1,1,0,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
000002.jpg,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
000003.jpg,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
000004.jpg,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
000005.jpg,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202595.jpg,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1
202596.jpg,0,0,0,0,0,1,1,0,0,1,...,0,1,1,0,0,0,0,0,0,1
202597.jpg,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1


In [9]:

# Настройки
IMAGE_SIZE = 128
BATCH_SIZE = 64
EPOCHS = 5
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Трансформации изображений
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Класс для работы с датасетом
class CelebADataset(Dataset):
    def __init__(self, image_dir, attributes, transform=None):
        self.image_dir = image_dir
        self.attributes = attributes
        self.transform = transform
        self.image_names = attributes.index.tolist()
        self.labels = attributes.values

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label



In [10]:
# Создание датасета и DataLoader'ов
dataset = CelebADataset(IMAGES_PATH, attributes, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Определение модели
class PersonalityNet(nn.Module):
    def __init__(self, num_classes):
        super(PersonalityNet, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return torch.sigmoid(self.model(x))

model = PersonalityNet(40).to(DEVICE)

# Определение функции потерь и оптимизатора
criterion = nn.BCELoss()  # Поскольку задача многоклассовой классификации
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)



C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Функции обучения и тестирования
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, test_loader, device):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            all_predictions.append(outputs.cpu())
            all_labels.append(labels.cpu())
    return torch.cat(all_predictions), torch.cat(all_labels)

# Обучение модели
for epoch in range(EPOCHS):
    train_model(model, train_loader, criterion, optimizer, DEVICE)
    print(f"Epoch {epoch + 1}/{EPOCHS} completed")

# Оценка модели
predictions, labels = evaluate_model(model, test_loader, DEVICE)
print("Модель обучена. Примерные предсказания готовы.")


Epoch 1/5 completed
Epoch 2/5 completed
Epoch 3/5 completed
Epoch 4/5 completed
Epoch 5/5 completed
Модель обучена. Примерные предсказания готовы.


In [7]:
model.save(model.state_dict(), "first.pth")

AttributeError: 'PersonalityNet' object has no attribute 'save'

In [9]:
torch.save(model.state_dict(), "personality_net.pth")


In [11]:
from PIL import Image
import torch
from torchvision import transforms

# Путь к сохранённой модели и изображению
MODEL_SAVE_PATH = os.path.join(DATASET_PATH, "personality_net.pth")
TEST_IMAGE_PATH = os.path.join(DATASET_PATH, "test.jpg")

# Загрузка сохранённой модели
loaded_model = PersonalityNet(40).to(DEVICE)
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
loaded_model.eval()

# Трансформации, используемые для тестового изображения
test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Загрузка и предобработка изображения
image = Image.open(TEST_IMAGE_PATH).convert("RGB")
input_tensor = test_transform(image).unsqueeze(0).to(DEVICE)  # Добавляем batch-измерение

# Прогнозирование
with torch.no_grad():
    prediction = loaded_model(input_tensor)
    prediction = prediction.cpu().numpy().flatten()  

print("Предсказание модели для изображения:")
print(prediction)


Предсказание модели для изображения:
[3.0175091e-03 1.1096884e-02 2.9316878e-01 1.1136906e-02 2.1512981e-03
 2.8176941e-03 1.3565101e-01 2.3516755e-01 3.5564315e-01 1.0284782e-02
 3.4893269e-03 5.7684337e-03 1.9923749e-03 4.9963924e-03 4.2326874e-04
 1.1037186e-02 6.3004591e-02 7.2231865e-03 6.3887229e-03 1.1533502e-03
 9.6195430e-01 6.2320475e-03 1.5536165e-01 3.0114928e-02 6.1908370e-01
 4.2426363e-02 3.4213507e-01 1.9881774e-01 7.4257322e-02 4.2065021e-05
 8.3094416e-03 7.1551907e-04 2.7483240e-01 1.8896520e-02 8.7727304e-04
 7.8109135e-03 1.4488252e-02 4.1198338e-04 7.4729431e-01 6.6049969e-01]


In [7]:
attributes.columns.tolist()

['5_o_Clock_Shadow',
 'Arched_Eyebrows',
 'Attractive',
 'Bags_Under_Eyes',
 'Bald',
 'Bangs',
 'Big_Lips',
 'Big_Nose',
 'Black_Hair',
 'Blond_Hair',
 'Blurry',
 'Brown_Hair',
 'Bushy_Eyebrows',
 'Chubby',
 'Double_Chin',
 'Eyeglasses',
 'Goatee',
 'Gray_Hair',
 'Heavy_Makeup',
 'High_Cheekbones',
 'Male',
 'Mouth_Slightly_Open',
 'Mustache',
 'Narrow_Eyes',
 'No_Beard',
 'Oval_Face',
 'Pale_Skin',
 'Pointy_Nose',
 'Receding_Hairline',
 'Rosy_Cheeks',
 'Sideburns',
 'Smiling',
 'Straight_Hair',
 'Wavy_Hair',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Lipstick',
 'Wearing_Necklace',
 'Wearing_Necktie',
 'Young']

In [18]:
attribute_names = attributes.columns.tolist()  

decoded_predictions = {attribute: prob for attribute, prob in zip(attribute_names, prediction)}

positive_attributes = {attr: prob for attr, prob in decoded_predictions.items() if prob > 0.3}

print("Полные предсказания:")
print(decoded_predictions)

print("\nВероятные положительные атрибуты (prob > 0.5):")
print(positive_attributes)


Полные предсказания:
{'5_o_Clock_Shadow': 0.0030175091, 'Arched_Eyebrows': 0.011096884, 'Attractive': 0.29316878, 'Bags_Under_Eyes': 0.011136906, 'Bald': 0.002151298, 'Bangs': 0.002817694, 'Big_Lips': 0.135651, 'Big_Nose': 0.23516755, 'Black_Hair': 0.35564315, 'Blond_Hair': 0.010284782, 'Blurry': 0.003489327, 'Brown_Hair': 0.0057684337, 'Bushy_Eyebrows': 0.001992375, 'Chubby': 0.0049963924, 'Double_Chin': 0.00042326874, 'Eyeglasses': 0.011037186, 'Goatee': 0.06300459, 'Gray_Hair': 0.0072231865, 'Heavy_Makeup': 0.006388723, 'High_Cheekbones': 0.0011533502, 'Male': 0.9619543, 'Mouth_Slightly_Open': 0.0062320475, 'Mustache': 0.15536165, 'Narrow_Eyes': 0.030114928, 'No_Beard': 0.6190837, 'Oval_Face': 0.042426363, 'Pale_Skin': 0.34213507, 'Pointy_Nose': 0.19881774, 'Receding_Hairline': 0.07425732, 'Rosy_Cheeks': 4.206502e-05, 'Sideburns': 0.008309442, 'Smiling': 0.00071551907, 'Straight_Hair': 0.2748324, 'Wavy_Hair': 0.01889652, 'Wearing_Earrings': 0.00087727304, 'Wearing_Hat': 0.0078109135